# TCG Card Detection Training

Train YOLOv8-nano for card detection using synthetic training data.

**Prerequisites:**
- Card images in Google Drive
- Optional: Background images for synthetic scene generation

**Estimated Time:** ~4-6 hours for full training

## 1. Setup Environment

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Install dependencies
!pip install -q ultralytics pyyaml tqdm opencv-python pillow

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set paths
DRIVE_PROJECT = '/content/drive/MyDrive/tcg-scanner'
WORK_DIR = '/content/tcg-scanner'

import os
os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Copy data from Drive (if not already in work dir)
import shutil
from pathlib import Path

# Copy card images
cards_src = Path(DRIVE_PROJECT) / 'ml/data/images/riftbound'
cards_dst = Path(WORK_DIR) / 'data/images/riftbound'

if cards_src.exists() and not cards_dst.exists():
    print("Copying card images from Drive...")
    shutil.copytree(cards_src, cards_dst)
    print(f"Copied to {cards_dst}")
else:
    print(f"Cards directory: {cards_dst} (exists: {cards_dst.exists()})")

# Count images
if cards_dst.exists():
    num_images = sum(1 for _ in cards_dst.rglob('*.jpg'))
    print(f"Found {num_images} card images")

## 2. Generate Synthetic Detection Data

Create training scenes by compositing cards onto backgrounds.

In [ ]:
# Synthetic scene generator (simplified version for Colab)
import cv2
import numpy as np
import random
from pathlib import Path
from tqdm import tqdm

class SyntheticSceneGenerator:
    def __init__(self, cards_dir, output_size=(640, 640)):
        self.cards_dir = Path(cards_dir)
        self.output_size = output_size
        self.card_paths = list(self.cards_dir.rglob('*.jpg'))
        print(f"Loaded {len(self.card_paths)} card images")
    
    def _get_background(self):
        """Generate synthetic background."""
        h, w = self.output_size
        colors = [
            (139, 90, 43), (169, 130, 91), (64, 64, 64),
            (45, 45, 45), (20, 60, 20), (30, 30, 80),
            (80, 80, 80), (200, 200, 200), (240, 230, 220),
        ]
        color = random.choice(colors)
        color = tuple(max(0, min(255, c + random.randint(-20, 20))) for c in color)
        bg = np.full((h, w, 3), color, dtype=np.uint8)
        
        # Add noise
        noise = np.random.normal(0, 10, bg.shape).astype(np.int16)
        bg = np.clip(bg.astype(np.int16) + noise, 0, 255).astype(np.uint8)
        return bg
    
    def generate_scene(self):
        """Generate a scene with 1-3 cards."""
        h, w = self.output_size
        scene = self._get_background()
        annotations = []
        
        num_cards = random.randint(1, 3)
        
        for _ in range(num_cards):
            card_path = random.choice(self.card_paths)
            card = cv2.imread(str(card_path))
            if card is None:
                continue
            
            # Random scale
            scale = random.uniform(0.2, 0.5)
            card_h, card_w = card.shape[:2]
            new_w = int(w * scale)
            new_h = int(new_w * card_h / card_w)
            card = cv2.resize(card, (new_w, new_h))
            
            # Random rotation
            angle = random.uniform(-30, 30)
            center = (new_w // 2, new_h // 2)
            M = cv2.getRotationMatrix2D(center, angle, 1.0)
            cos, sin = abs(M[0, 0]), abs(M[0, 1])
            rot_w = int(new_h * sin + new_w * cos)
            rot_h = int(new_h * cos + new_w * sin)
            M[0, 2] += (rot_w - new_w) / 2
            M[1, 2] += (rot_h - new_h) / 2
            card = cv2.warpAffine(card, M, (rot_w, rot_h), borderValue=(0, 0, 0))
            
            # Random position
            card_h, card_w = card.shape[:2]
            x = random.randint(0, max(0, w - card_w))
            y = random.randint(0, max(0, h - card_h))
            
            # Place card on scene
            x1, y1 = x, y
            x2 = min(x + card_w, w)
            y2 = min(y + card_h, h)
            
            card_x1 = 0
            card_y1 = 0
            card_x2 = x2 - x1
            card_y2 = y2 - y1
            
            # Simple blend (black pixels are transparent)
            mask = (card[card_y1:card_y2, card_x1:card_x2] > 10).all(axis=2)
            scene[y1:y2, x1:x2][mask] = card[card_y1:card_y2, card_x1:card_x2][mask]
            
            # Calculate bounding box (YOLO format)
            cx = (x1 + x2) / 2 / w
            cy = (y1 + y2) / 2 / h
            bw = (x2 - x1) / w
            bh = (y2 - y1) / h
            
            if bw > 0.05 and bh > 0.05:  # Filter tiny boxes
                annotations.append((0, cx, cy, bw, bh))
        
        return scene, annotations
    
    def generate_dataset(self, output_dir, num_train=5000, num_val=1000):
        """Generate complete YOLO dataset."""
        output_dir = Path(output_dir)
        
        for split, num in [('train', num_train), ('val', num_val)]:
            images_dir = output_dir / split / 'images'
            labels_dir = output_dir / split / 'labels'
            images_dir.mkdir(parents=True, exist_ok=True)
            labels_dir.mkdir(parents=True, exist_ok=True)
            
            print(f"\nGenerating {split} set ({num} images)...")
            for i in tqdm(range(num)):
                scene, annotations = self.generate_scene()
                
                cv2.imwrite(str(images_dir / f'{i:05d}.jpg'), scene)
                
                with open(labels_dir / f'{i:05d}.txt', 'w') as f:
                    for ann in annotations:
                        f.write(f'{ann[0]} {ann[1]:.6f} {ann[2]:.6f} {ann[3]:.6f} {ann[4]:.6f}\n')
        
        # Create data.yaml
        data_yaml = f"""path: {output_dir.absolute()}
train: train/images
val: val/images
names:
  0: card
nc: 1
"""
        with open(output_dir / 'data.yaml', 'w') as f:
            f.write(data_yaml)
        
        print(f"\nDataset saved to {output_dir}")
        print(f"Data config: {output_dir / 'data.yaml'}")

In [ ]:
# Generate synthetic dataset
generator = SyntheticSceneGenerator(
    cards_dir=Path(WORK_DIR) / 'data/images/riftbound',
    output_size=(640, 640)
)

generator.generate_dataset(
    output_dir=Path(WORK_DIR) / 'data/detection',
    num_train=5000,  # Reduce for faster testing, increase to 10000 for full training
    num_val=1000
)

In [ ]:
# Visualize some generated scenes
import matplotlib.pyplot as plt

detection_dir = Path(WORK_DIR) / 'data/detection/train/images'
sample_images = list(detection_dir.glob('*.jpg'))[:6]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
for ax, img_path in zip(axes.flat, sample_images):
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ax.imshow(img)
    ax.set_title(img_path.name)
    ax.axis('off')
plt.tight_layout()
plt.show()

## 3. Train YOLOv8

In [ ]:
from ultralytics import YOLO

# Initialize model
model = YOLO('yolov8n.pt')  # nano model for mobile

# Train
results = model.train(
    data=str(Path(WORK_DIR) / 'data/detection/data.yaml'),
    epochs=100,  # Reduce for testing
    imgsz=640,
    batch=32,
    optimizer='AdamW',
    lr0=0.001,
    weight_decay=0.0005,
    warmup_epochs=5,
    patience=20,
    project=str(Path(WORK_DIR) / 'runs/detection'),
    name='card_detector',
    exist_ok=True,
    # Augmentation
    hsv_h=0.02,
    hsv_s=0.8,
    hsv_v=0.5,
    degrees=30,
    translate=0.15,
    scale=0.6,
    shear=10,
    perspective=0.002,
    flipud=0.0,
    fliplr=0.5,
    mosaic=0.9,
    mixup=0.2,
)

In [ ]:
# Validate
metrics = model.val()
print(f"\nValidation Results:")
print(f"  mAP50: {metrics.box.map50:.4f}")
print(f"  mAP50-95: {metrics.box.map:.4f}")

## 4. Export Model

In [ ]:
# Export to CoreML (iOS)
model.export(format='coreml', imgsz=640, int8=True)

# Export to TFLite (Android)
model.export(format='tflite', imgsz=640, int8=True)

In [ ]:
# Copy model to Drive
run_dir = Path(WORK_DIR) / 'runs/detection/card_detector'
drive_models = Path(DRIVE_PROJECT) / 'models/detection'
drive_models.mkdir(parents=True, exist_ok=True)

# Copy best model
shutil.copy(run_dir / 'weights/best.pt', drive_models / 'best.pt')
print(f"Model saved to Drive: {drive_models / 'best.pt'}")

## 5. Test Detection

In [ ]:
# Test on some images
test_images = list((Path(WORK_DIR) / 'data/detection/val/images').glob('*.jpg'))[:4]

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
for ax, img_path in zip(axes.flat, test_images):
    results = model(str(img_path))
    annotated = results[0].plot()
    ax.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    ax.axis('off')
plt.tight_layout()
plt.show()